In [ ]:
!export HF_DATASETS_OFFLINE=1
!export HF_HUB_OFFLINE=1

In [1]:
from transformers import AutoImageProcessor, DonutSwinModel
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image", trust_remote_code=True)
image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained("nielsr/donut-base")
model = DonutSwinModel.from_pretrained("./donut_encoder")

inputs = image_processor([image,image], return_tensors="pt")
inputs
# with torch.no_grad():
#     outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
# list(last_hidden_states.shape)

{'pixel_values': tensor([[[[ 0.8667,  0.8667,  0.8588,  ...,  0.0980,  0.0980,  0.0980],
          [ 0.8667,  0.8667,  0.8588,  ...,  0.0980,  0.0980,  0.0980],
          [ 0.8745,  0.8745,  0.8667,  ...,  0.1059,  0.1059,  0.1059],
          ...,
          [-0.4275, -0.4275, -0.4118,  ..., -0.2000, -0.1922, -0.1922],
          [-0.4275, -0.4275, -0.4118,  ..., -0.2078, -0.2000, -0.2000],
          [-0.4275, -0.4275, -0.4118,  ..., -0.2078, -0.2000, -0.2000]],

         [[-0.2157, -0.2157, -0.2314,  ..., -0.8039, -0.8039, -0.8039],
          [-0.2157, -0.2157, -0.2314,  ..., -0.8039, -0.8039, -0.8039],
          [-0.2078, -0.2078, -0.2235,  ..., -0.8039, -0.8039, -0.8039],
          ...,
          [-0.8588, -0.8588, -0.8667,  ..., -0.9137, -0.9216, -0.9216],
          [-0.8667, -0.8667, -0.8745,  ..., -0.9216, -0.9216, -0.9216],
          [-0.8667, -0.8667, -0.8745,  ..., -0.9216, -0.9216, -0.9216]],

         [[ 0.3725,  0.3725,  0.4196,  ..., -0.5608, -0.5608, -0.5608],
          [ 0

In [ ]:
[image,image]

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>]

: 

In [3]:
model.cuda()
inputs = inputs.to(torch.device("cuda"))
with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

OutOfMemoryError: CUDA out of memory. Tried to allocate 300.00 MiB. GPU 0 has a total capacity of 1.95 GiB of which 13.62 MiB is free. Including non-PyTorch memory, this process has 1.93 GiB memory in use. Of the allocated memory 1.86 GiB is allocated by PyTorch, and 20.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [18]:
visual_embedding = outputs.pooler_output

In [2]:
import torchvision

In [3]:
image2 = torchvision.transforms.Resize((1000,1000))(image)

In [4]:
inputs = image_processor(image2, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

[1, 4800, 1024]

In [7]:
image_processor

DonutImageProcessor {
  "do_align_long_axis": true,
  "do_normalize": true,
  "do_pad": true,
  "do_rescale": true,
  "do_resize": true,
  "do_thumbnail": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "DonutImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "processor_class": "DonutProcessor",
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 2560,
    "width": 1920
  }
}

In [ ]:
model()

DonutSwinModel(
  (embeddings): DonutSwinEmbeddings(
    (patch_embeddings): DonutSwinPatchEmbeddings(
      (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    )
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): DonutSwinEncoder(
    (layers): ModuleList(
      (0): DonutSwinStage(
        (blocks): ModuleList(
          (0): DonutSwinLayer(
            (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attention): DonutSwinAttention(
              (self): DonutSwinSelfAttention(
                (query): Linear(in_features=128, out_features=128, bias=True)
                (key): Linear(in_features=128, out_features=128, bias=True)
                (value): Linear(in_features=128, out_features=128, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): DonutSwinSelfOutput(
                (dense): Linear(in

In [9]:
outputs.pooler_output.shape

torch.Size([1, 1024])

In [ ]:
import re

from transformers import AutoProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch

processor = AutoProcessor.from_pretrained("naver-clova-ix/donut-base")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

device = "cpu"
model.to(device)
# load document image
dataset = load_dataset("hf-internal-testing/example-documents", split="test")
image = dataset[1]["image"]

# prepare decoder inputs
task_prompt = "Text hihkjnfsdj"
inputs = processor(images=image, text=task_prompt,return_tensors="pt")
# decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

# pixel_values = processor(image, return_tensors="pt").pixel_values
with torch.no_grad():
    outputs = model(pixel_values = inputs.pixel_values.to(device),decoder_input_ids=inputs.input_ids, output_hidden_states=True)
outputs

In [ ]:
from transformers import AutoProcessor, VisionEncoderDecoderModel
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

In [ ]:
model.encoder

In [ ]:
from transformers import DonutSwinModel
encoder = DonutSwinModel.from_pretrained("nielsr/donut-base")

In [ ]:
encoder

In [ ]:
model.encoder.state_dict() == encoder.state_dict()

In [ ]:
encoder.state_dict()

In [ ]:
import torch
for n, p in model.encoder.named_parameters():
    print(torch.nn.functional.mse_loss(p.data,encoder.state_dict()[n].data))
    # if (p.data != encoder.state_dict()[n].data):
    
    #     print(n)

In [ ]:
model.encoder.save_pretrained("donut_encoder")

In [34]:
from transformers import DonutSwinModel
encoder = DonutSwinModel.from_pretrained("./donut_encoder")

In [19]:
from transformers import AutoTokenizer, XLMRobertaModel
import torch

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
model = XLMRobertaModel.from_pretrained("FacebookAI/xlm-roberta-base")

inputs = tokenizer("Hello, my cat is cute", return_tensors="pt")
outputs = model(**inputs)

textual_embedding = outputs.pooler_output

In [40]:
inputs

{'input_ids': tensor([[    0, 35378,     4,   759,  7515,    83, 99942,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
last_hidden_states.shape

torch.Size([1, 8, 768])

In [13]:
outputs.pooler_output.shape

torch.Size([1, 768])

In [38]:
model

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine

In [15]:
model.pooler

XLMRobertaPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [27]:
mhsa = torch.nn.MultiheadAttention(embed_dim=1024,num_heads=4,kdim=768, batch_first=True)
fused = mhsa(visual_embedding.unsqueeze(1),textual_embedding.unsqueeze(1),visual_embedding.unsqueeze(1))

In [28]:
fused[0].shape

torch.Size([1, 1, 1024])

In [44]:
class FusionLayer(torch.nn.Module):
    def __init__(self, visual_dim, text_dim, num_heads=4):
        super().__init__()
        self.visual_dim = visual_dim
        self.text_dim = text_dim
        self.mhsa = torch.nn.MultiheadAttention(self.visual_dim,num_heads,kdim=text_dim,batch_first=True)
        self.fc = torch.nn.Sequential(torch.nn.Linear(self.visual_dim,self.visual_dim*2),torch.nn.GELU(),torch.nn.Linear(self.visual_dim*2,self.visual_dim))
        self.layer_norm = torch.nn.LayerNorm(self.visual_dim)

    def forward(self, visual_embedding, textual_embedding):
        visual_embedding = visual_embedding.unsqueeze(1)
        textual_embedding = textual_embedding.unsqueeze(1)
        fused_embed,_ = self.mhsa(visual_embedding,textual_embedding,visual_embedding)
        normalize_fused_embed = self.layer_norm(fused_embed.squeeze(1)) + visual_embedding.squeeze(1)
        layer_embed = self.layer_norm(self.fc(normalize_fused_embed)) + normalize_fused_embed
        return layer_embed
    
class FusionAttentionModule(torch.nn.Module):
    def __init__(self, visual_dim, text_dim, num_layers = 3):
        super().__init__()
        self.fusion_layers = [FusionLayer(visual_dim,text_dim) for _ in range(num_layers)]
    
    def forward(self, visual_embedding, textual_embedding):
        for layer in self.fusion_layers:
            visual_embedding = layer(visual_embedding,textual_embedding)
        return visual_embedding

    
class ComponentEncoder(torch.nn.Module):
    def __init__(self,num_layers=3,freeze_visual=True, freeze_textual=True):
        super().__init__()
        self.visual_encoder = DonutSwinModel.from_pretrained("./donut_encoder")
        self.textual_encoder = XLMRobertaModel.from_pretrained("FacebookAI/xlm-roberta-base")
        if freeze_visual:
            for p in self.visual_encoder.parameters():
                p.requires_grad = False
        if freeze_textual:
            for p in self.textual_encoder.parameters():
                p.requires_grad = False
        self.fusion_module = FusionAttentionModule(self.visual_encoder.config.hidden_size,self.textual_encoder.config.hidden_size,num_layers)

    def forward(self, image_inputs, text_inputs):
        visual_embedding = self.visual_encoder(**image_inputs).pooler_output
        textual_embedding = self.textual_encoder(**text_inputs).pooler_output
        fused_embedding = self.fusion_module(visual_embedding,textual_embedding)
        return fused_embedding


In [45]:
FusionLayer(1024,768)(visual_embedding,textual_embedding).shape

torch.Size([1, 1024])

In [37]:
encoder.config.hidden_size

1024

In [39]:
model.config

XLMRobertaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "FacebookAI/xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [46]:
component_encoder = ComponentEncoder(num_layers=1)
image_inputs = image_processor(image, return_tensors="pt")
text_inputs = tokenizer("Hello, my cat is cute", return_tensors="pt")
component_embed = component_encoder(image_inputs, text_inputs)

In [47]:
component_embed.shape

torch.Size([1, 1024])

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
text_inputs = tokenizer(["Hello, my cat is cute","Hello, my jvdj rat  mat dat is cute"], return_tensors="pt", padding=True)

In [4]:
text_inputs

{'input_ids': tensor([[    0, 35378,     4,   759,  7515,    83, 99942,     2,     1,     1,
             1,     1,     1],
        [    0, 35378,     4,   759,  1647, 59805,   170, 12540,  2589,   607,
            83, 99942,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
import torch
text_inputs.to(torch.device("cuda"))

{'input_ids': tensor([[    0, 35378,     4,   759,  7515,    83, 99942,     2,     1,     1,
             1,     1,     1],
        [    0, 35378,     4,   759,  1647, 59805,   170, 12540,  2589,   607,
            83, 99942,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [8]:
a = torch.rand(2,4).cuda()

In [9]:
a

tensor([[0.6810, 0.9874, 0.4391, 0.1518],
        [0.7820, 0.4890, 0.9823, 0.8956]], device='cuda:0')

In [12]:
torch.log_softmax(a,dim=1).exp()

tensor([[0.2679, 0.3639, 0.2103, 0.1578],
        [0.2446, 0.1825, 0.2989, 0.2740]], device='cuda:0')

In [ ]:
torch.nn.Softmax(dim=1)(a)

tensor([[0.2679, 0.3639, 0.2103, 0.1578],
        [0.2446, 0.1825, 0.2989, 0.2740]], device='cuda:0')

: 